## Import dependencies

In [1]:
import cv2
import urllib.request
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import time
import cv2
import urllib.request
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip



## Define crawl function

In [2]:

def CrawlVNExpress(article_url):
    url = article_url
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    title = soup.title.string
    print("Tiêu đề bài báo:", title)
    article_content = soup.find("article", class_="fck_detail")
    if article_content:
        paragraphs = article_content.find_all("p")
        article_text = "\n".join([p.get_text() for p in paragraphs])
        print("Nội dung bài báo:\n", article_text)
    else:
        print("Không tìm thấy nội dung bài báo.")
    image_tags = soup.find_all("img", itemprop="contentUrl")
    image_urls = [img.get("data-src") for img in image_tags]
    for idx, image_url in enumerate(image_urls):
        print(f"Hình ảnh {idx + 1}: {image_url}")
    return article_text, image_urls

def TTS(text):
    url = "https://viettelai.vn/tts/speech_synthesis"
    TOKEN = ''
    if not TOKEN:
        raise Exception("Token is empty")

    payload = json.dumps({
        "text": text,
        "voice": "hn-phuongtrang",
        "speed": 1,
        "tts_return_option": 3,
        "token": "836d310c50fc6b920867ce47bee06201",
        "without_filter": False
    })
    headers = {
        'accept': '*/*',
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    with open("audio.mp3", "wb") as file:
        file.write(response.content)
    return "audio.mp3"


def AskGemini(text):
    print('Start asking Gemini...')
    api_key = ''
    if not api_key:
        raise Exception("Token is empty")
    url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent'
    headers = {
        'Content-Type': 'application/json'
    }

    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": "Đọc lại bài báo sau thành đúng 50 từ: \n" + text
                    }
                ]
            }
        ]
    }
    response = requests.post(f'{url}?key={api_key}',
                            headers=headers, json=data)
    if response.status_code == 200:
        response_data = response.json()
        if 'candidates' in response_data and len(response_data['candidates']) > 0:
            candidate = response_data['candidates'][0]
            text_output = candidate['content']['parts'][0]['text']
            print('Output Text:', text_output)
            return text_output
    else:
        print('Error:', response.status_code, response.text)


def download_and_save(url):
    download_count = 0
    while download_count < 20:
        try:
            response = requests.get(url)
            status_code = response.status_code
            if status_code != 200:
                raise Exception("Failed to download, status code:", status_code)
            file_data = response.content
            file_name = url.split("/")[-1]
            with open(file_name,"wb") as file:
                file.write(file_data)
            print("Downloaded", file_name)
            return file_name
        except Exception as e:
            print("Download failed:", e)
            download_count += 1
            time.sleep(2)


def download_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image




## Define video processing function

In [3]:

def create_video_from_images(imgfile, audio_url, video_dim=(1280, 720), fps=27,
                             start_center=(0.4, 0.6), end_center=(0.5, 0.5),
                             start_scale=0.9, end_scale=1.0, output_file="output.mp4"):

    # Tải clip âm thanh và lấy độ dài
    audio_clip = AudioFileClip(audio_url)
    duration = audio_clip.duration
    audio_clip.close()

    # Tính toán thời gian cho mỗi ảnh
    num_images = len(imgfile)
    duration_per_image = duration / num_images

    print("Thời gian cho mỗi ảnh: ", duration_per_image)
    print("Số lượng ảnh: ", num_images)
    print("Tổng thời gian: ", duration)

    # Tải ảnh
    img = [download_image(file) for file in imgfile]
    orig_shape = img[0].shape[:2]

    # Easing function để tạo hiệu ứng mượt mà hơn
    def easing(t):
        return t**3 * (t * (t * 6 - 15) + 10)  # Smoothstep easing function

    def crop(image, x, y, w, h):
        """
        Crop hình ảnh, đảm bảo không vượt ra ngoài biên và thêm padding nếu cần.
        """
        height, width = image.shape[:2]

        # Xác định biên crop
        x0 = max(0, int(x - w / 2))
        y0 = max(0, int(y - h / 2))
        x1 = min(width, int(x + w / 2))
        y1 = min(height, int(y + h / 2))

        # Crop ảnh
        cropped = image[y0:y1, x0:x1]

        # Nếu crop nhỏ hơn kích thước yêu cầu, thêm padding
        if cropped.shape[0] != h or cropped.shape[1] != w:
            padded = np.zeros((h, w, 3), dtype=np.uint8)  # Tạo khung đen
            offset_y = (h - cropped.shape[0]) // 2
            offset_x = (w - cropped.shape[1]) // 2
            padded[offset_y:offset_y + cropped.shape[0],
                   offset_x:offset_x + cropped.shape[1]] = cropped
            return padded

        return cropped

    # Tạo khung hình
    frames = []
    for i in range(num_images):
        for frame_index in range(int(fps * duration_per_image)):
            # Áp dụng easing
            alpha = easing(frame_index / (fps * duration_per_image))
            rx = end_center[0] * alpha + start_center[0] * (1 - alpha)
            ry = end_center[1] * alpha + start_center[1] * (1 - alpha)
            x = int(orig_shape[1] * rx)
            y = int(orig_shape[0] * ry)
            scale = end_scale * alpha + start_scale * (1 - alpha)

            # Tính toán kích thước ảnh
            if orig_shape[1] / orig_shape[0] > video_dim[0] / video_dim[1]:
                h = int(orig_shape[0] * scale)
                w = int(h * video_dim[0] / video_dim[1])
            else:
                w = int(orig_shape[1] * scale)
                h = int(w * video_dim[1] / video_dim[0])

            cropped = crop(img[i], x, y, w, h)
            scaled = cv2.resize(cropped, dsize=video_dim,
                                interpolation=cv2.INTER_CUBIC)  # Nội suy mượt hơn
            frames.append(scaled)

    # Ghi video với âm thanh
    # video_clip = ImageSequenceClip(
    #     [cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames], fps=fps)
    # video_clip = video_clip.set_audio(AudioFileClip(audio_url))
    # video_clip.write_videofile(
    #     'out_' + output_file, codec="libx264", audio_codec="aac")

    # Export mp4 video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, video_dim)
    for frame in frames:
        out.write(frame)
    out.release()

    import moviepy.editor as mpe
    my_clip = mpe.VideoFileClip(output_file)
    audio_background = mpe.AudioFileClip(audio_url)
    final_clip = my_clip.set_audio(audio_background)
    final_clip.write_videofile('outnameee.mp4', fps=fps)

    print("Video đã được tạo thành công!")


In [4]:

def crawler(article_url):
    print("Crawling article from:", article_url)
    article_text, image_urls = CrawlVNExpress(article_url)
    sumarized_text = AskGemini(article_text)
    audio_url = TTS(sumarized_text)
    # audio_saved_path = download.download_and_save(audio_url)
    create_video_from_images(
        image_urls, audio_url, output_file="audio.mp4")

article_url = input(
    'Nhap link bai bao VNExpress\n VD: https://vnexpress.net/ba-ngay-o-ha-giang-dau-dong-4826655.html')
# article_url = 'https://vnexpress.net/nhieu-xe-container-o-tp-hcm-lap-camera-xoa-diem-mu-tranh-tai-nan-4817333.html'
crawler(article_url)


Crawling article from: https://vnexpress.net/ba-ngay-o-ha-giang-dau-dong-4826655.html
Tiêu đề bài báo: Ba ngày ở Hà Giang đầu đông - Báo VnExpress Du lịch
Nội dung bài báo:
 Khung cảnh Hà Giang trên cung đường từ Đồng Văn đến Mèo Vạc, được nhiếp ảnh gia Vũ Ngọc Thiện chụp vào chiều 2/12. Anh Thiện dành ba ngày đầu tháng lên Hà Giang trải nghiệm cảnh sắc thiên nhiên vào đông. Đây là chuyến đi đầu tiên của anh đến tỉnh địa đầu tổ quốc.
Du khách đến từ Thái Bình xuất phát hôm 1/12, theo lịch trình Hà Nội - Đồng Văn - sông Nho Quế - Mèo Vạc - Lũng Cú - Lô Lô Chải - dốc Thẩm Mã - TP Hà Giang rồi quay về Hà Nội.
Khung cảnh Hà Giang trên cung đường từ Đồng Văn đến Mèo Vạc, được nhiếp ảnh gia Vũ Ngọc Thiện chụp vào chiều 2/12. Anh Thiện dành ba ngày đầu tháng lên Hà Giang trải nghiệm cảnh sắc thiên nhiên vào đông. Đây là chuyến đi đầu tiên của anh đến tỉnh địa đầu tổ quốc.
Du khách đến từ Thái Bình xuất phát hôm 1/12, theo lịch trình Hà Nội - Đồng Văn - sông Nho Quế - Mèo Vạc - Lũng Cú - Lô Lô

MoviePy - Done.
Moviepy - Writing video outnameee.mp4



Moviepy - Done !
Moviepy - video ready outnameee.mp4
Video đã được tạo thành công!
